In [13]:
import pandas as pd
from ftplib import FTP
from datetime import datetime
import time
import numpy as np
import pymongo
from pymongo import MongoClient

from geopy import distance

import get_recent_days as gtdys
import muni_etl
import labelling

Connect to the database
Connect to a specific GTFS-period collection

In [22]:
client = MongoClient('localhost', 27017)

db = client['testavl']
coll = db['clean']

Get all unique blocks in the database

In [31]:
blocks = coll.distinct('TRAIN_ASSIGNMENT')

Get a sample from the collection to get a sample datetime object

In [32]:
sample = coll.find_one()
raw_date = sample['REPORT_TIME']
cln_date = datetime.strptime(raw_date, '%m/%d/%Y %H:%M:%S')

Use the datetime object to look up the proper gtfs data

In [33]:
gtfs_lookup_df = pd.read_csv('data/gtfs_lookup.csv')
gtfs_directory = gtdys.get_gtfs_file(cln_date, gtfs_lookup_df)

Knowing the proper gtfs data, load in trips, schedules, stops and the calendar, filtering them based on the blocks 

First, convert the blocks to ints for referencing

In [49]:
int_blocks = [int(blk) for blk in blocks]
int_blocks

[3301, 3302, 3303, 3304, 3305, 3306, 3307]

Trips

In [60]:
trip_txt = 'data/gtfs/{}/trips.txt'.format(gtfs_directory)
trips = pd.read_csv(trip_txt)
trip_blocks = trips[trips['block_id'].isin(int_blocks)]
trip_ids = trip_blocks['trip_id'].unique()
trip_ids.shape

(358,)

Schedule

In [61]:
sched_txt = 'data/gtfs/{}/stop_times.txt'.format(gtfs_directory)
sched = pd.read_csv(sched_txt)
sched_trps = sched[sched['trip_id'].isin(trip_ids)]
sched_trps.shape

(15026, 9)

In [56]:
sched.sample(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
225796,7239030,11:41:59,11:41:59,5625,33,,,,


In [70]:
sched['departure_time'].max()

'30:34:00'

In [71]:
print('\a')




Get a random date from the data, convert to datetime

In [4]:
sample = coll.find_one()
raw_date = sample['REPORT_TIME']
cln_date = datetime.strptime(raw_date, '%m/%d/%Y %H:%M:%S')

Use the datetime to lookup the correct GTFS directory

In [5]:
gtfs_lookup_df = pd.read_csv('data/gtfs_lookup.csv')
gtfs_directory = gtdys.get_gtfs_file(cln_date, gtfs_lookup_df)

Get trip_id's that match our day, block and service

In [7]:
spec_trips = trips[(trips['block_id'] == 3302) & (trips['service_id'] == 3) & (trips['direction_id'] == 0)]
trip_ids = spec_trips['trip_id'].unique()

Get the trip schedule

In [8]:
stptm_txt = 'data/gtfs/{}/stop_times.txt'.format(gtfs_directory)
stp_tm = pd.read_csv(stptm_txt)

Get the id of the first schedule stop of our filtered trips

In [9]:
stp_tm_trips= stp_tm[stp_tm['trip_id'].isin(trip_ids)]
start_times = stp_tm_trips[stp_tm_trips['stop_sequence'] == 1]
start_stop_id = start_times.sample()['stop_id'].values[0]

Get the stops

In [10]:
stop_txt = 'data/gtfs/{}/stops.txt'.format(gtfs_directory)
stops = pd.read_csv(stop_txt)

Get the latitude and longitude of our starting stop

In [19]:
start_stop = stops[stops['stop_id'] == start_stop_id]
srtstp_latlon = (start_stop['stop_lat'].values[0], start_stop['stop_lon'].values[0])

Get all rows that get within 20 meters of our stop

In [99]:
beginnings = []

for doc in coll.find():

    doc_latlon = (doc['LATITUDE'], doc['LONGITUDE'])
    
    if distance.distance(srtstp_latlon, doc_latlon).m <= 20:
        
        if doc['TRAIN_ASSIGNMENT'] == '3302':
        
            beginnings.append(doc)

Cluster those rows by their temporal proximity

In [100]:
start_time_clusters = {}

# For each item in starting-stop intersections
for idx, item in enumerate(beginnings):
    
    # Convert the reported time to a datetime object
    cln_date = datetime.strptime(item['REPORT_TIME'], '%m/%d/%Y %H:%M:%S')
 
    # If we are just starting, there are no start-clusters. Create one!
    if len(start_time_clusters) == 0:
        
        start_time_clusters[idx] = [item]
        
    else:  
        
        # Let's check if this row is close to any of the start clusters in our dictionary 
        matched = 0
        
        # For each start cluster in the dictionary
        for key, value in start_time_clusters.items():
            
            # For each row (reported momement) in the time cluster
            for time in value:
                
                # Get the time difference between the two moments
                time_diff = cln_date - datetime.strptime(time['REPORT_TIME'], '%m/%d/%Y %H:%M:%S')
                
                # If these two rows occured within 2 minutes of each other
                if time_diff.seconds < 240:
                    
                    # This row belongs in the cluster! Add it, break the loop, add to 'matched'
                    start_time_clusters[key].append(item)
                    matched +=1
                    
                    break
        
        # If this row doesn't match with any existing clusters
        if matched == 0:

            # Make a new cluster for it!
            start_time_clusters[idx] = [item]

Let's inspect the min and max times of each start cluster we found:

In [101]:
for key, value in start_time_clusters.items():
    
    times = []
    
    for item in value:
        
        times.append(datetime.strptime(item['REPORT_TIME'], '%m/%d/%Y %H:%M:%S'))
        
    print ("Range: {} to {}".format(min(times), max(times)))

Range: 2016-11-19 17:15:23 to 2016-11-19 17:28:53
Range: 2016-11-19 19:31:53 to 2016-11-19 19:49:53
Range: 2016-11-19 06:35:32 to 2016-11-19 06:49:02
Range: 2016-11-19 08:09:37 to 2016-11-19 08:18:37
Range: 2016-11-19 10:08:46 to 2016-11-19 10:28:16
Range: 2016-11-19 12:36:35 to 2016-11-19 12:50:05


Finally, let's compare to these to the scheduled start times of our filtered trips:

In [102]:
start_times.sort_values('departure_time')

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
489402,7253963,06:50:00,06:50:00,6293,1,,,,
489534,7253966,08:20:00,08:20:00,6293,1,,,,
489226,7253959,10:30:00,10:30:00,6293,1,,,,
488882,7253951,12:50:00,12:50:00,6293,1,,,,
488574,7253944,15:10:00,15:10:00,6293,1,,,,
488266,7253937,17:30:00,17:30:00,6293,1,,,,
487958,7253930,19:50:00,19:50:00,6293,1,,,,


It's a match!

Okay, nice and clean above.    
Let's add a timestamp to each database doc:

In [103]:
for doc in coll.find():
    
    object_id = doc['_id']
    cln_date = datetime.strptime(doc['REPORT_TIME'], '%m/%d/%Y %H:%M:%S')
    
    time_stamp = cln_date.timestamp()
    
    coll.update_one({'_id':object_id}, {"$set": {'time_stamp':time_stamp}})

In [104]:
coll.find_one()

{'_id': ObjectId('5ae20f933ad39e1a4fce277c'),
 'REV': '1526',
 'REPORT_TIME': '11/19/2016 00:00:51',
 'VEHICLE_TAG': '5401',
 'LONGITUDE': '-122.41817',
 'LATITUDE': '37.76513',
 'SPEED': '3.889',
 'HEADING': '90.0',
 'TRAIN_ASSIGNMENT': '3303',
 'PREDICTABLE': '1',
 'time_stamp': 1479542451.0}

In [65]:
coll.find({ 'time_stamp': { '$exists': True}}).count()

8365

In [66]:
coll.find().count()

8365

Hooray, Timestamp!

Okay, now let's add an index...

In [105]:
coll.create_index([("TRAIN_ASSIGNMENT", 1), ("time_stamp", 1)])

'TRAIN_ASSIGNMENT_1_time_stamp_1'

In [106]:
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'testavl.clean'},
 'TRAIN_ASSIGNMENT_1_time_stamp_1': {'v': 2,
  'key': [('TRAIN_ASSIGNMENT', 1), ('time_stamp', 1)],
  'ns': 'testavl.clean'}}

cool!

Now, let's try a good 'ol test filtering?

In [107]:
test_row = coll.find_one({'REPORT_TIME':'11/19/2016 17:28:53'})
start_ts = test_row['time_stamp']
start_train = test_row['TRAIN_ASSIGNMENT']

In [108]:
test_grab = coll.find({'time_stamp': {"$gt": start_ts}, 'TRAIN_ASSIGNMENT': start_train}).sort("time_stamp").limit(5)

In [109]:
for doc in test_grab:
    print(doc['REPORT_TIME'])

11/19/2016 17:29:59
11/19/2016 17:30:23
11/19/2016 17:31:29
11/19/2016 17:31:53
11/19/2016 17:32:32


Sweet! I have a way to loop through every row after a 'start'

### Let's step back and join our starts with our scheduled starts...

In [110]:
starts = []

for key, value in start_time_clusters.items():
    
    times = []
    
    best_time = {'time_stamp':0}
    
    for item in value:
        
        if item['time_stamp'] > best_time['time_stamp']:
            
            best_time = item
            
    starts.append(best_time)
    

In [186]:
scheduled_starts = start_times['departure_time']
parsed_starts = scheduled_starts.apply(lambda x: datetime.strptime(x, '%H:%M:%S'))


for item in starts:
    
    raw_time = (item['REPORT_TIME'].split(" ")[1])
    rawtime_parse = datetime.strptime(raw_time, '%H:%M:%S')
        
    diff_df = ballin.apply(lambda x: abs(x-rawtime_parse))
    
    sched_start = start_times.loc[diff_df.idxmin()]
    
    time_diff = diff_df.min().seconds
    
    
    start_details = {}
    start_details['trip_id'] = int(sched_start['trip_id'])
    start_details['sched_time_diff_seconds'] = time_diff
    start_details['trip_start'] = 1

    
    cln_date = datetime.fromtimestamp(item['time_stamp'])
    iso = cln_date.strftime('%Y-%m-%d')
    start_details['trip_id_iso'] = str(sched_start['trip_id']) + '_' + iso
  

    coll.update_one({'_id':item['_id']}, {"$set": start_details})


In [187]:
coll.find_one({ 'trip_id_iso': { '$exists': True}})

{'_id': ObjectId('5ae20f943ad39e1a4fce2949'),
 'REV': '1526',
 'REPORT_TIME': '11/19/2016 17:28:53',
 'VEHICLE_TAG': '5411',
 'LONGITUDE': '-122.45646',
 'LATITUDE': '37.78697',
 'SPEED': '0.0',
 'HEADING': '0.0',
 'TRAIN_ASSIGNMENT': '3302',
 'PREDICTABLE': '1',
 'time_stamp': 1479605333.0,
 'trip_id': 7253937,
 'trip_id_iso': '7253937_2016-11-19',
 'sched_time_diff_seconds': 67,
 'trip_start': 1}

FUCK YEA!    
TIME TO LOOP!

I got the station_id of my 'end' stop yesterday:

My ending stop_id is 3289      
It’s Lat/Lon is (37.765377, -122.413290)

In [211]:
end_stop = stops[stops['stop_id'] == 3289]
endstp_latlon = (end_stop['stop_lat'].values[0], end_stop['stop_lon'].values[0])

In [224]:
for start in coll.find({ 'trip_start': 1}):
    
    print ("Trip Tag: ", start['trip_id_iso'])
    print ("Start Time: ", start['REPORT_TIME'])
    
    str_timestamp = start['time_stamp']
    str_block = start['TRAIN_ASSIGNMENT']
    str_vehicle = start['VEHICLE_TAG']
    
    search = {}
    
    search['TRAIN_ASSIGNMENT'] = str_block
    search['VEHICLE_TAG'] = str_vehicle
    search['time_stamp'] = {"$gt": str_timestamp}
    
    count = 0
    breakin = 0
    
    trip_rows = []
    
    for data in coll.find(search).sort('time_stamp'):
        
        trip_rows.append(data['_id'])
        
        data_latlon = (data['LATITUDE'], data['LONGITUDE'])

        if distance.distance(endstp_latlon, data_latlon).m <= 20:
            
            print("Stop Time", data['REPORT_TIME'])
            
            breakin += 1
            
            break
        
        count += 1
        
    if breakin == 0:

        print ("No End!")
        print ("Last row: ")
        print (data)
        
    else:
        
        tripid_iso = start['trip_id_iso']
        
        for row in trip_rows:
            
            coll.update_one({'_id':row}, {"$set": {'trip_id_iso': tripid_iso}})
            
            
        
    print ("Row count between start and end: ", count)
    print ("\n")
        
        
    
        

Trip Tag:  7253937_2016-11-19
Start Time:  11/19/2016 17:28:53
Stop Time 11/19/2016 18:28:53
Row count between start and end:  81


Trip Tag:  7253930_2016-11-19
Start Time:  11/19/2016 19:49:53
Stop Time 11/19/2016 20:40:53
Row count between start and end:  79


Trip Tag:  7253963_2016-11-19
Start Time:  11/19/2016 06:49:02
Stop Time 11/19/2016 07:28:02
Row count between start and end:  72


Trip Tag:  7253966_2016-11-19
Start Time:  11/19/2016 08:18:37
Stop Time 11/19/2016 09:05:46
Row count between start and end:  69


Trip Tag:  7253959_2016-11-19
Start Time:  11/19/2016 10:28:16
Stop Time 11/19/2016 11:22:16
Row count between start and end:  79


Trip Tag:  7253951_2016-11-19
Start Time:  11/19/2016 12:50:05
Stop Time 11/19/2016 13:51:35
Row count between start and end:  83




# Alright! Let's inspect some trips that NEVER END

In [213]:
import folium

map_2 = folium.Map(location=[37.770373, -122.436064],
                   tiles='Stamen Terrain',
                   zoom_start=13)

for start in coll.find({ 'trip_id_iso': '7253937_2016-11-19'}):
    
    str_timestamp = start['time_stamp']
    str_block = start['TRAIN_ASSIGNMENT']
    str_vehicle = start['VEHICLE_TAG']
    
    search = {}
    
    search['TRAIN_ASSIGNMENT'] = str_block
    search['VEHICLE_TAG'] = str_vehicle
    search['time_stamp'] = {"$gt": str_timestamp}
    
    count = 0
    breakin = 0
    
    for data in coll.find(search).sort('time_stamp'):
        lat = data['LATITUDE']
        lon = data['LONGITUDE']

        rprt_time = data['REPORT_TIME']

        dist = distance.distance(endstp_latlon, (float(lat), float(lon))).m

        folium.Marker([float(lat), float(lon)], popup=rprt_time + " + " + str(dist)).add_to(map_2)



folium.Marker([endstp_latlon[0], endstp_latlon[1]], icon=folium.Icon(color='red')).add_to(map_2)
    
map_2

# Okay, let's try this again, with a last stop lookup...

In [225]:
for start in coll.find({ 'trip_start': 1}):
    
    print ("Trip Tag: ", start['trip_id_iso'])
    print ("Start Time: ", start['REPORT_TIME'])
    
    str_timestamp = start['time_stamp']
    str_block = start['TRAIN_ASSIGNMENT']
    str_vehicle = start['VEHICLE_TAG']
    
    search = {}
    
    search['TRAIN_ASSIGNMENT'] = str_block
    search['VEHICLE_TAG'] = str_vehicle
    search['time_stamp'] = {"$gt": str_timestamp}
    
    trip_id = start['trip_id']
    trip_sched = stp_tm[stp_tm['trip_id'] == trip_id]
    lst_stop_id = trip_sched.tail(1)['stop_id'].values[0]
  
    end_stop = stops[stops['stop_id'] == lst_stop_id]
    endstp_latlon = (end_stop['stop_lat'].values[0], end_stop['stop_lon'].values[0])

    count = 0
    breakin = 0
    
    trip_rows = []
    
    for data in coll.find(search).sort('time_stamp'):
        
        trip_rows.append(data['_id'])
        
        data_latlon = (data['LATITUDE'], data['LONGITUDE'])

        if distance.distance(endstp_latlon, data_latlon).m <= 20:
            
            print("Stop Time", data['REPORT_TIME'])
            
            breakin += 1
            
            break
        
        count += 1
        
    if breakin == 0:

        print ("No End!")
        print ("Last row: ")
        print (data)
        
    else:
        
        tripid_iso = start['trip_id_iso']
        
        for row in trip_rows:
            
            coll.update_one({'_id':row}, {"$set": {'trip_id_iso': tripid_iso}})
            
            
        
    print ("Row count between start and end: ", count)
    print ("\n")

Trip Tag:  7253937_2016-11-19
Start Time:  11/19/2016 17:28:53
Stop Time 11/19/2016 18:28:53
Row count between start and end:  81


Trip Tag:  7253930_2016-11-19
Start Time:  11/19/2016 19:49:53
Stop Time 11/19/2016 20:40:53
Row count between start and end:  79


Trip Tag:  7253963_2016-11-19
Start Time:  11/19/2016 06:49:02
Stop Time 11/19/2016 07:28:02
Row count between start and end:  72


Trip Tag:  7253966_2016-11-19
Start Time:  11/19/2016 08:18:37
Stop Time 11/19/2016 09:05:46
Row count between start and end:  69


Trip Tag:  7253959_2016-11-19
Start Time:  11/19/2016 10:28:16
Stop Time 11/19/2016 11:22:16
Row count between start and end:  79


Trip Tag:  7253951_2016-11-19
Start Time:  11/19/2016 12:50:05
Stop Time 11/19/2016 13:51:35
Row count between start and end:  83




In [229]:
coll.find({'trip_id_iso': '7253937_2016-11-19'}).count()

83